In [1]:
N = 32

class Server:
    """для описания работы серверов в сети
    Соответственно в объектах класса Server должны быть локальные свойства:
    buffer - список принятых пакетов (изначально пустой);
    ip - IP-адрес текущего сервера. Уникальный IP-адрес каждого сервера 
    должен формироваться автоматически при создании нового экземпляра класса Server.
    """
    servers = []
    
    def __new__(cls):
        if len(cls.servers) < N: 
            obj = super().__new__(cls)
            cls.servers.append(obj)
            setattr(obj, 'ip', len(cls.servers))
            return obj
        else:
            print('Превышено общее число серверов')
    
    def __init__(self):
        self._buffer = []
        
    def send_data(self, data):
        """для отправки информационного пакета data (объекта класса Data) 
        с указанным IP-адресом получателя (пакет отправляется роутеру и 
        сохраняется в его буфере - локальном свойстве buffer);
        """
      #  print(f'Отправка сообщения {data.data} на сервер {data.ip}')
        self.router.buffer.append(data)

    def get_data(self):
        """возвращает список принятых пакетов (если ничего принято не было, 
        то возвращается пустой список) и очищает входной буфер;
        """
        res = self._buffer.copy()
        self._buffer = []
        return res

    def get_ip(self):
        """возвращает свой IP-адрес.
        """
        return self.ip


class Router:
    """для описания работы роутеров в сети (в данной задаче полагается один роутер).
    И одно обязательное локальное свойство (могут быть и другие свойства):
    buffer - список для хранения принятых от серверов пакетов (объектов класса Data).
    """
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
        
    def __init__(self):
        self.links = []
        self.buffer = []
        
    def link(self, server):
        """для присоединения сервера server (объекта класса Server) к роутеру
        """
       # print(f'привязка сервера {server.ip} к роутеру')
        self.links.append(server)
        server.router = self
       # print(f'всего приязано {len(self.links)}')
    
    def unlink(self,server):
        """для отсоединения сервера server (объекта класса Server) от роутера
        """
       # print(f'отвязка сервера {server.ip} от роутера')
        self.links.remove(self, server)
    
    def send_data(self):
        """для отправки всех пакетов (объектов класса Data) из буфера роутера 
        соответствующим серверам (после отправки буфер должен очищаться)
        """
        #print(f'сообщений в буфере роутера {len(self.buffer)}')
        for msg in self.buffer:
            for server in self.links:
                if msg.ip == server.get_ip():
                    server._buffer.append(msg.clone())
                    #print(f'Роутер отправил {msg} на сервер {server.get_ip()}')
        self.buffer = []

    

class Data:
    """для описания пакета информации
    Наконец, объекты класса Data должны содержать, два следующих локальных свойства:
    data - передаваемые данные (строка);
    ip - IP-адрес назначения.
    """
    def __init__(self, data, ip):
        self.data = data
        self.ip = ip
    
    def clone(self):
        obj = super().__new__(Data)
        obj.__dict__ = self.__dict__.copy()
        return obj    

In [2]:
router = Router()
sv_from = Server()
router.link(sv_from)
router.link(Server())
router.link(Server())
sv_to = Server()
router.link(sv_to)
sv_from.send_data(Data("Hello", sv_to.get_ip()))
#router.send_data()
sv_to.send_data(Data("Hi", sv_from.get_ip()))
router.send_data()
msg_lst_from = sv_from.get_data()
msg_lst_to = sv_to.get_data()

In [3]:
print(msg_lst_to[0].data, len(msg_lst_to))
print(msg_lst_from[0].data, len(msg_lst_from))

Hello 1
Hi 1


In [ ]:
router.send_data()


In [ ]:
print(router.buffer)
print()
for ser in Server.servers:
    print(ser._buffer)

In [ ]:
assert msg_lst_from[0].data == "Hi" and msg_lst_to[0].data == "Hello", "данные не прошли по сети, классы не функционируют должным образом"

In [ ]:
router = Router()
sv_from = Server()
router.link(sv_from)
router.link(Server())
router.link(Server())
sv_to = Server()
router.link(sv_to)
sv_from.send_data(Data("Hello", sv_to.get_ip()))
sv_to.send_data(Data("Hi", sv_from.get_ip()))
router.send_data()
msg_lst_from = sv_from.get_data()
msg_lst_to = sv_to.get_data()

assert msg_lst_from[0].data == "Hi" and msg_lst_to[0].data == "Hello", "данные не прошли по сети, классы не функционируют должным образом"